In [1]:
import numpy as np
import json
import pretty_midi as pm
import sys
import mido

from tiv import *

# MIDI processing

In [2]:
def midifile_to_dict(mid):
    tracks = []
    for track in mid.tracks:
        tracks.append([vars(msg).copy() for msg in track])
    return {
        'ticks_per_beat': mid.ticks_per_beat,
        'tracks': tracks,
    }

def get_bpm(file_name):
    mid = mido.MidiFile(file_name)
    dict = midifile_to_dict(mid)
    tempo = dict['tracks'][0][0]['tempo']
    bpm = 0

    if (bpm == 0):
        bpm = mido.tempo2bpm(tempo)
    else:
        tempo = mido.bpm2tempo(bpm)

    return bpm

In [3]:
with open('chord_set/json/bar8_list.json', 'r') as bl:
    bar8_list = json.load(bl)

In [4]:
bar8_list[0]

'what-hurts-the-most_intro'

In [ ]:
a = pm.PrettyMIDI('midi/what-hurts-the-most_intro.mid')
bpm = get_bpm('midi/what-hurts-the-most_intro.mid')
bpm

In [ ]:
a.get_piano_roll(fs=bpm/60).shape

# get 8-bar chroma data (in format .npy), size: 32*12

In [ ]:
bar8_data = []

for fn in bar8_list:
    a = pm.PrettyMIDI('midi/'+fn+'.mid')
    bpm = get_bpm('midi/'+fn+'.mid')
    #print(bpm)
#     if a.get_chroma(fs=bpm/60+0.001).shape != (12,32):
#         print(0)
    #print(a.get_chroma(fs=bpm/60+0.001).shape)
    
    bar8_data.append((a.get_chroma(fs=bpm/60+0.001)/100).transpose())

In [ ]:
bar8_np = np.stack(bar8_data)

In [ ]:
np.save('bar8_cp_np.npy', bar8_np)

In [14]:
b = np.load('data/bar8_cp_np.npy')

In [15]:
b[0].shape

(32, 12)

In [6]:
bar8_with_quality_data = []

for fn in bar8_list:
    a = pm.PrettyMIDI('midi/'+fn+'.mid')
    bpm = get_bpm('midi/'+fn+'.mid')
    
    #print(bpm)
#     if a.get_chroma(fs=bpm/60+0.001).shape != (12,32):
#         print(0)
    #print(a.get_chroma(fs=bpm/60+0.001).shape)
    
    bar8_with_quality_data.append((a.get_chroma(fs=bpm/60+0.001)/100).transpose())

In [7]:
bar8_with_quality_np = np.stack(bar8_with_quality_data)

In [8]:
np.save('bar8_cp_with_quality_np.npy', bar8_with_quality_np)


In [ ]:
a.get_piano_roll(fs=bpm/60).shape

# get 8-bar chroma data (in format .npy), size: 32*12

In [ ]:
bar8_data = []

for fn in bar8_list:
    a = pm.PrettyMIDI('midi/'+fn+'.mid')
    bpm = get_bpm('midi/'+fn+'.mid')
    #print(bpm)
#     if a.get_chroma(fs=bpm/60+0.001).shape != (12,32):
#         print(0)
    #print(a.get_chroma(fs=bpm/60+0.001).shape)
    
    bar8_data.append((a.get_chroma(fs=bpm/60+0.001)/100).transpose())

In [ ]:
bar8_np = np.stack(bar8_data)

In [ ]:
np.save('bar8_cp_np.npy', bar8_np)

In [ ]:
b = np.load('bar8_cp_np.npy')

In [ ]:
b[0] == bar8_np[0]

100.0

In [11]:
a.get_piano_roll(fs=bpm/60).shape

(128, 32)

# get 8-bar chroma data (in format .npy), size: 32*12

In [12]:
bar8_data = []

for fn in bar8_list:
    a = pm.PrettyMIDI('midi/'+fn+'.mid')
    bpm = get_bpm('midi/'+fn+'.mid')
    #print(bpm)
#     if a.get_chroma(fs=bpm/60+0.001).shape != (12,32):
#         print(0)
    #print(a.get_chroma(fs=bpm/60+0.001).shape)
    
    bar8_data.append((a.get_chroma(fs=bpm/60+0.001)/100).transpose())

In [13]:
bar8_np = np.stack(bar8_data)

In [14]:
np.save('bar8_cp_np.npy', bar8_np)

In [15]:
b = np.load('bar8_cp_np.npy')

In [18]:
b[0] == bar8_np[0]

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True

# TIV processing

In [118]:
def get_tiv_polar(chroma):
    '''
    input: 12 dims chroma
    return: 12 dims TIV in polar coordinate, first 6 dims--mags, last 6 dims-- phases
    '''
    tiv = TIV.from_pcp(chroma)
    tiv_polar = np.concatenate((tiv.mags(), tiv.phases()))
    
    return tiv_polar

def get_tiv_polar_list(chroma_list):
    tiv_list = chroma_list.copy()
    for i in range(chroma_list.shape[0]):
        tiv_list[i][:] = get_tiv_polar(chroma_list[i][:])
    return tiv_list

def chroma2tiv_np(chroma_chunk):
    tiv_chunk = chroma_chunk.copy()
    for i in range(chroma_chunk.shape[0]):
        tiv_chunk[i] = get_tiv_polar_list(chroma_chunk[i])
    return tiv_chunk

In [122]:
bar8_cp_tiv_np = chroma2tiv_np(bar8_cp_np)
bar8_cp_wq_tiv_np = chroma2tiv_np(bar8_cp_wq_np)

In [123]:
np.save('bar8_cp_tiv_np.npy', bar8_cp_tiv_np)
np.save('bar8_cp_wq_tiv_np.npy', bar8_cp_wq_tiv_np)